In [22]:
import numpy as np
import matplotlib.pyplot as plt
import clustering as cl
import util
import healpy as hp
import importlib as imp
from collections import defaultdict

%matplotlib inline 

In [2]:
# Global parameters
gs = [0.4]
gdots = [-0.004, -0.002, 0.0, 0.002, 0.004]
g_gdots = [(x,y) for x in gs for y in gdots]

nside=8
n = -14
dt=15.0
cr=0.00124
pixels=range(hp.nside2npix(nside))

train_path = 'demo_train/UnnObs_Training_1_line_A_ec_labelled_2457308.5_pm15.0_r2.5.trans'
itf_path = 'demo_itf/itf_new_1_line_ec_2457308.5_pm15.0_r2.5.trans'


In [9]:
true_count_set, mergedCounter_dict, mergedTime_dict = cl.accessible_clusters(pixels, infilename=train_path)
true_count = len(true_count_set)
print('True count of clusters: {}'.format(true_count))

right, wrong, ids_right, ids_wrong = cl.test_clusters(pixels, train_path, util.lunation_center(n), \
                                                    dt=dt,rad=cr)

print('Using our optimal parameters we got {0} percent of clusters with {1} percent errors.'.format(right/true_count,wrong/true_count))
print('We got',right,'right and',wrong,'wrong out of total',true_count)

True count of clusters: 890
Starting run...
100%

Run finished!
Using our optimal parameters we got 0.7842696629213484 percent of clusters with 0.0056179775280898875 percent errors.
We got 698 right and 5 wrong out of total 890


In [18]:
tr_results, tr_clust_ids = cl.find_clusters([281], train_path, util.lunation_center(n), g_gdots=g_gdots,dt=dt,rad=cr)

Starting run...
100%

Run finished!


In [41]:
# tr_clust_ids

In [27]:
cl = imp.reload(cl)
res_dict = cl.test_things(train_path,[281],nside,n)

In [29]:
agg_dict = defaultdict(list)
for k,v in tr_clust_ids.items():
    # k is the tracklet id, v is the cluster id 
    agg_dict[v].append(res_dict[k])

In [40]:
# agg_dict

In [ ]:
g,gdot = 0.4,0.0
results = []
for k, v in agg_dict.items():
    cx, mx, cy, my, t0 = full_fit_trkl(t_ref, g, gdot, v)
    result = (k, cx, mx, cy, my, t0)
    results.append(result)



In [34]:
a_obs3 = [itm for ob in agg_dict[3] for itm in ob]

In [37]:
print('We have {} observations in cluster id 3'.format(len(a_obs3)))

We have 16 observations in cluster id 3


In [91]:
from scipy.optimize import curve_fit
def getXval():
    a = 4; b = 3; c = 1;
    f0 = a*b**2*c
    f1 = a*b/c
    return np.array([f0, f1])

def fit(x, a0, a1):
    res = a0*x[0] + a1*x[1]
    return np.array([res])

x = getXval()
y = np.array([0.15, 0.34])
popt, pcov = curve_fit(fit, x, y)
print(popt, pcov)

[-0.32652778  1.        ] [[ inf  inf]
 [ inf  inf]]


/Users/paulblankley/anaconda3/lib/python3.6/site-packages/scipy/optimize/minpack.py:785: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [123]:
g,gdot = 0.4,0.0
cl = imp.reload(cl)
par3 = cl.full_fit_t_loss(util.lunation_center(n),g,gdot,a_obs3)

init [0.053674328369746643, 0.0043643417901996304, -0.14487208480303945, 0.00011781505504137331, 0.4, 0.0]
params [  5.36942886e-02   4.35762057e-03  -1.44925312e-01   1.32662286e-04
   4.00008738e-01   1.46824314e-05]
diff [1.9960267360022432e-05, 6.7212246068742293e-06, 5.3227425767021774e-05, 1.4847231433794738e-05, 8.7382360245968904e-06, 1.468243143533497e-05]
everything       fun: 1.167487984018433e-08
 hess_inv: array([[  3.87669529e-02,  -6.10970957e-03,  -5.74508931e-03,
          2.06934839e-02,  -2.19311149e-02,  -8.54251883e-02],
       [ -6.10970957e-03,   9.57165671e-03,   5.21635782e-03,
         -2.18737041e-02,   1.94621166e-02,   9.03539237e-02],
       [ -5.74508931e-03,   5.21635782e-03,   8.09615229e-02,
          1.07816445e-03,   2.18929180e-01,   5.93265986e-03],
       [  2.06934839e-02,  -2.18737041e-02,   1.07816445e-03,
          5.48474280e-02,   8.88409866e-04,  -2.25145981e-01],
       [ -2.19311149e-02,   1.94621166e-02,   2.18929180e-01,
          8.884

In [124]:
par3

array([  5.36942886e-02,   4.35762057e-03,  -1.44925312e-01,
         1.32662286e-04,   4.00008738e-01,   1.46824314e-05])

In [ ]:
# cust loss